In [1]:
from dl import authClient as ac, queryClient as qc, storeClient as sc, helpers
import pylab as plt
import numpy as np
from astropy.io import fits
import matplotlib as mpl
import math
import astropy.io.fits as fits
from astropy.table import Table, Column
import scipy.optimize as sp

In [2]:
token=ac.login('mgkadrmas')

In [3]:
df=Table(fits.getdata('LMCParallax4.fits'))
df['Dist']=1/df['parallax']
df['Disterror']=df['sigma']/(df['parallax']**2)

In [4]:
def onclick(event):
    print('%s click: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          ('double' if event.dblclick else 'single', event.button,
           event.x, event.y, event.xdata, event.ydata))
    
    
    
def roi_cut(xcut,ycut,x,y): 
    from matplotlib.path import Path
    tupVerts = list(zip(xcut,ycut))
    points = np.vstack((x,y)).T
    p = Path(tupVerts) # make a polygon
    inside = p.contains_points(points)
    ind, = np.where(~inside)
    cutind, = np.where(inside)
    return ind, cutind
    print (ind, cutind)

In [5]:
xcut=[102.237903,99.022177,95.584677,90.372984,85.715726,75.625000,71.189516,67.752016,63.981855,62.207661,59.879032,58.770161,57.217742,56.219758,59.213710,63.870968,71.522177,78.840726,87.046371,93.256048,100.020161,103.014113,104.012097]
ycut=[-64.938814,-63.151940,-61.439520,-59.801552,-58.312490,-58.238037,-58.461396,-59.429286,-60.173817,-62.109597,-65.385533,-68.512562,-71.490685,-75.138886,-76.851306,-77.893650,-78.489274,-78.191462,-78.191462,-77.521384,-75.362245,-72.458575,-68.438109]
x=df['ra'].ravel()
y=df['dec'].ravel()
(ind, cutind) = roi_cut(xcut,ycut,x,y)


In [6]:
def InclineDiskDist(ra, dec, pa, io, d0):
    ra1 = np.copy(ra)
    dec1 = np.copy(dec)
    ra1 = np.radians(ra1)
    dec1 = np.radians(dec1)
    alph0 = np.radians(81.9) #np.radians(82.25) #ra of LMC center
    delt0 = np.radians(-69.866667) #np.radians(-69.5) #dec of LMC center
    sd = np.sin(delt0)
    cd = np.cos(delt0)
    io = np.radians(io) #inclination #25.86
    pa = np.radians(pa) #np.radians(149.37+90)  #position angle of line-of-nodes
    cosr = cd*np.cos(dec1)*np.cos(ra1-alph0)+sd*np.sin(dec1)
    sinrcosph = -np.cos(dec1)*np.sin(ra1-alph0)
    sinrsinph = cd*np.sin(dec1)-sd*np.cos(dec1)*np.cos(ra1-alph0)
    dist = d0*np.cos(io)/(np.cos(io)*cosr-np.sin(io)*np.cos(pa)*sinrsinph+np.sin(io)*np.sin(pa)*sinrcosph)
    return dist


In [7]:
def creatediskmodel(theta, inclin, Distcenter):
    dtype=np.dtype([('ra',float),('dec',float),('distheory',float)])
    cat = np.zeros([56,26], dtype=dtype)
    for x in range (56):
        ra=x+55
        for y in range (26):
            dec=y-80
            (distheory) = InclineDiskDist(ra,dec,theta,inclin,Distcenter)
            cat[x,y]['ra']=ra
            cat[x,y]['dec']=dec
            cat[x,y]['distheory']=distheory
    return cat

In [8]:
x0=np.array([235,25,50],dtype=float)

def findbestdiskmodel(cat):
    def residuals(x):
        model=creatediskmodel(x[0], x[1], x[2])
        return (cat['Dist'].ravel()[cutind] - model['distheory'].ravel()[cutind])
    soln=sp.least_squares(residuals, x0)
    theta = soln.x[0]
    inclin = soln.x[1]
    Distcenter = soln.x[2]
    return theta, inclin, Distcenter

In [9]:
findbestdiskmodel(df)

(235.2767305108276, 52.75566708462706, 48.92247898198689)

In [31]:
dtype=np.dtype([('theta',float),('inclin',float),('distcenter',float)])
MC = np.zeros(100, dtype=dtype)

model = creatediskmodel(235.276, 52.75566, 48.92247)

for t in range(100):
    synth = df.copy()
    synth['Dist'] = model['distheory'] + np.random.randn(model.shape[0],model.shape[1])*df['Disterror']
    synth['Disterror'] = df['Disterror']
    theta, inclin, distcenter = findbestdiskmodel(synth)
    MC[t]['theta'] = theta
    MC[t]['inclin'] = inclin
    MC[t]['distcenter'] = distcenter

MC = Table(MC)
MC.write('LMCPlxMCbig4.fits', format='fits')

KeyboardInterrupt: 

In [15]:
MC=fits.getdata('LMCPlxMCbig4.fits')

In [16]:
MC['theta']

array([344.58147492, 501.31208075, 456.51646489, 164.51475703,
       343.8764891 , 525.17913162, 166.3463124 , 342.83734342,
       164.11398835, 345.35623135, 525.85069379, 520.18498336,
       342.63821765, 165.56318161, 526.92238517, 353.06569237,
       164.38413868, 345.24171996, 348.70379098, 188.48724986,
       164.27624508, 344.53144598, 165.75301761,  42.73813385,
       326.67065468, -15.06846751, 343.58463864, 452.29319348,
       522.73114294, 522.55614971, 164.95977637, 346.10000485,
       344.70463499, 345.13533725, 523.35695445, 347.10625749,
       346.2597407 , -14.57806474, -15.50325291, 164.44290273,
       523.58600093, 164.44761812, 344.51802144, -14.8070694 ,
        37.4882918 , 344.22644512, 164.00550809, -10.57930232,
       523.70942696, 524.21143956, 343.39059278,  -6.95713252,
       346.72174095, 343.69371269, 164.25823199, 343.19298101,
       435.03020981, -13.29861583, 344.50995386, 261.55383709,
       342.99263609, 164.6214088 , 344.50556791, 252.69

In [17]:
MC['inclin']

array([  77.47199055,  -74.24306288,  101.7306037 ,  102.39569164,
         77.41687578,  102.42503977,  282.36710226, -102.65720739,
        102.51415048, -102.52356422,  -77.63624929,  -77.13227733,
       -102.66951495,  102.40976936,  282.37472689,   79.90989584,
        102.4854159 , -282.38701305, -102.41955402,   31.2503682 ,
        102.44199496, -102.53771432,  102.42932523,  103.41800366,
         74.74979037,  257.62685878,   77.16910351,   99.81504144,
        -77.08615097,  -77.23905263,  282.40959744, -102.57480884,
       -102.59483557, -102.59549246,  102.61977816,  257.66688976,
       -102.55740982,  257.68398046,  257.62368101,  -77.53177509,
        102.42028082,  282.42189639, -102.55571029,  257.67849628,
        106.43123955,  257.39001442,  102.45063316,  104.00850589,
        -79.6585513 ,  102.41327041,   77.37592987,  259.96535626,
         77.37836399,   77.38424249,  102.7232658 ,   77.32638643,
        -81.31827167,   77.73594223, -102.7235662 ,   66.47323

In [18]:
np.std(MC['theta'])

167.7525237235683

In [19]:
np.std(MC['inclin'])

134.5295443281242

In [20]:
np.std(MC['distcenter'])

174.6638713179793